## Imports

In [2]:
import sys
sys.path.append('../')
import gc
import pandas as pd
import numpy as np

from src.fetch_data import DataLoader
from src.exploration import Analysis
from src.cleaning import CleanDataFrame
from src.visualization import Plotters


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default = "svg"


cleaner = CleanDataFrame()
analyzer = Analysis()
plotters = Plotters(w=6, h=4)

# pd.options.plotting.backend = 'matplotlib'
plt.rcParams.update({'font.size': 24})
# plt.rcParams.update({'xtick.labelsize': 'large'})
# plt.rcParams.update({'ytick.labelsize': 'large'})
plt.rcParams.update({'legend.fontsize': 24})
%matplotlib inline
# plt.rcParams.keys()



## Fetch the dataframes

In [3]:
# Then load the raw sales data
data_path = 'data/merged/train.csv'
version = 'merged_v1'
repo = '../'

train_df = DataLoader.dvc_get_data(data_path, version, repo)
train_df['Date'] = pd.to_datetime(train_df['Date'])
# Finally load the test data
data_path = 'data/merged/test.csv'
version = 'merged_v1'
repo = '../'

test_df = DataLoader.dvc_get_data(data_path, version, repo)
test_df['Date'] = pd.to_datetime(test_df['Date'])

DataLoaderLogger - INFO - DVC: CSV file read with path: data/merged/train.csv | version: merged_v1 | from: ../
DataLoaderLogger - INFO - DVC: CSV file read with path: data/merged/test.csv | version: merged_v1 | from: ../


# 

# Features correlation to target

In [7]:
train_df.corr()["Sales"]#.sort_values(ascending=False)


Store                        0.005126
DayOfWeek                   -0.462125
Sales                        1.000000
Customers                    0.894711
Open                         0.678472
Promo                        0.452345
SchoolHoliday                0.085124
CompetitionDistance         -0.018869
CompetitionOpenSinceMonth   -0.028257
CompetitionOpenSinceYear     0.012659
Promo2                      -0.091040
Promo2SinceWeek             -0.044143
Promo2SinceYear             -0.091056
Year                         0.023519
Month                        0.048768
WeekOfYear                   0.052946
SalesPerCustomer             0.186581
Name: Sales, dtype: float64

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         41088 non-null  int64         
 1   Store                      41088 non-null  int64         
 2   DayOfWeek                  41088 non-null  int64         
 3   Date                       41088 non-null  datetime64[ns]
 4   Open                       41077 non-null  float64       
 5   Promo                      41088 non-null  int64         
 6   StateHoliday               41088 non-null  object        
 7   SchoolHoliday              41088 non-null  int64         
 8   StoreType                  41088 non-null  object        
 9   Assortment                 41088 non-null  object        
 10  CompetitionDistance        41088 non-null  float64       
 11  CompetitionOpenSinceMonth  25872 non-null  float64       
 12  Comp

The number of Customer has the highes correlation with the Sales, but we don't have that in our test data. So, first I will filter the columns that are in my test data. 

In [8]:
test_colunns = set(test_df.columns)
train_columns = set(train_df.columns)
print(len(test_colunns), len(train_columns))
common_columns = test_colunns.intersection(train_columns)
print(len(common_columns))

17 22
16


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 22 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  object        
 8   SchoolHoliday              1017209 non-null  int64         
 9   StoreType                  1017209 non-null  object        
 10  Assortment                 1017209 non-null  object        
 11  CompetitionDistance        1017209 no

In [11]:
common_columns

{'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Date',
 'DayOfWeek',
 'Open',
 'Promo',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'PromoInterval',
 'SchoolHoliday',
 'StateHoliday',
 'Store',
 'StoreType'}

And these are the columns that are not included. Other than the Customer column, the other will be recreated next. I'm just removing them the data, to help me test the pre-processing pipeline I am building.

- Raw data comes in
- Feature engineering
- Droping columns
- Encoding categoricals
- Scalling


In [14]:
train_columns.difference(test_colunns)

{'Customers', 'Month', 'Sales', 'SalesPerCustomer', 'WeekOfYear', 'Year'}

In [ ]:
def generate_columns(df: pd.DataFrame) -> pd.DataFrame:
    assert "Date" in df.columns
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    df['is_month_end'] = df['Date'].dt.is_month_end
    df['is_month_start'] = df['Date'].dt.is_month_start
    df['is_quarter_end'] = df['Date'].dt.is_quarter_end
    df['is_quarter_start'] = df['Date'].dt.is_quarter_start
    df['is_year_end'] = df['Date'].dt.is_year_end
    df['is_year_start'] = df['Date'].dt.is_year_start
    


number of days to holiday,
number of days after holiday

These should be some type of cumilative sum and differnce

In [47]:
holidays = train_df.query("StateHoliday in ['a', 'b', 'c']")['Date'].dt.date.unique()
holidays.sort()


Now that I have the holiday dates, I need a function that takes in a date, then tells me it's distance on both dxns for the closest holiday

In [166]:
def get_neighbors(date):
    """uses a sorted list of dates to get the neighboring 
    dates for a date. 
    """
    date = pd.to_datetime(date)
    original_year = None
    if date.year >= holidays[-1].year:
        original_year = date.year
        # Assume the date given is in 2014
        date = pd.to_datetime(f"2014-{date.month}-{date.day}")
    previous, upcoming = None, None
    for i, d in enumerate(holidays):
        if d >= date.date():
            previous = holidays[i-1]
            upcoming = holidays[i]
            if original_year:
                print("Changed")
                previous = pd.to_datetime(
                    f"{original_year}-{previous.month}-{previous.day}")
                upcoming = pd.to_datetime(
                    f"{original_year}-{upcoming.month}-{upcoming.day}")
            else: print("Not change")
            return previous, upcoming


def get_holiday_distances(date) -> list[int, int]:
    """takes in a date, then tells me it's distance on both dxns for the closest holiday"""
    previous, upcoming = get_neighbors(date)
    try:
        after_holiday = date - previous
    except:
        print(previous, date, type(previous), type(date))
    to_next_holiday = upcoming - date
    
    return after_holiday, to_next_holiday


Now that we have our helper function, Let's use them an create the two new columns

Create the columns with empty value.

iterate over the unique dates and calculate the distances

Then replace the coresponding empiy values with the distances



In [167]:
def create_holiday_distance_cols(df: pd.DataFrame) -> pd.DataFrame:
    df['DistanceToNextHoliay'] = pd.NA
    df['DistanceFromPrevHoliay'] = pd.NA
    unique_dates = df.Date.unique()
    p = type(unique_dates[0])
    for date in unique_dates:
        after_holiday, to_next_holiday = get_holiday_distances(date)
        indecies = df[df['Date'] == date].index
        df.loc[indecies, 'DistanceToNextHoliay'] = to_next_holiday
        df.loc[indecies, 'DistanceFromPrevHoliay'] = after_holiday
        # break


In [168]:
preped = create_holiday_distance_cols(train_df)
    
# get_neighbors(my_date)

Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed
Changed


UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('O') and dtype('<M8[ns]')

In [108]:
get_neighbors(pd.to_datetime("2022-8-8").date())

new date 2014-08-08 00:00:00


(Timestamp('2022-06-19 00:00:00'), Timestamp('2022-10-03 00:00:00'))

In [93]:
holidays[-1].year == my_date.year

False